In [13]:
import numpy as np
import tensorflow as tf

In [14]:
def load_mnist():
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    nClass = 10
    x_train_flat = x_train.reshape(x_train.shape[0],-1).T
    x_test_flat = x_test.reshape(x_test.shape[0],-1).T
    y_train_onehot = np.eye(nClass)[y_train].T
    y_test_onehot = np.eye(nClass)[y_test]
    return x_train_flat, x_test_flat, y_train_onehot, y_test_onehot

In [15]:
x_train, x_test, y_train, y_test = load_mnist()

In [16]:
####### Utility Functions #########
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    return A

def relu(Z):
    A = np.maximum(0,Z)
    return A

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

In [17]:
##### Define Block ######
layers = [[784,'relu'],[50, 'relu'],[12, 'relu'],[45, 'relu'],[10,'relu']]

In [18]:
class deepfuzzy:
    W = dict()
    b = dict()
    caches = []
    grads = dict()
    AL = []
    def __init__(self,layers, x_train,x_test,y_train,y_test):
        self.layers = layers
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        self.L = len(layers)
    def initialize(self, initializer = 'random'):
        for i in range(1,self.L):
            self.W[str(i)] = np.random.randn(self.layers[i][0],self.layers[i-1][0])
            self.b[str(i)] = np.random.randn(self.layers[i][0],1)
    ###FORWARD PROP###
    @staticmethod
    def forward(A,W,b):
        Z = np.dot(W,A) + b
        cache = (A, W, b)
        return Z, cache
    @staticmethod        
    def forwardAct(A,W,b,activation = 'sigmoid'):
        Z, linear_cache = deepfuzzy.forward(A,W,b)
        activation_cache = Z
        if activation == 'sigmoid':
            A_next = sigmoid(Z)
        if activation == 'relu':
            A_next = relu(Z)
        return A_next, (linear_cache, activation_cache)
    
    def forwardProp(self):
        A = self.x_train
        for i in range (1,self.L):
            A, cache = deepfuzzy.forwardAct(A, self.W[str(i)], self.b[str(i)], activation = layers[i-1][1])
            self.caches.append(cache)
            print(A.shape)
        self.AL = A
###SAVE WEIGHTS###
    def saveWeights(self):
        np.save('W.npy',  self.W)  
        np.save('b.npy', self.b)
        my_dict_back = np.load('my_dict.npy')
        
    def loadWeights(self):
        self.W = np.load('W.npy')
        self.b = np.load('b.npy')
###BACKWARD PROP###
    @staticmethod    
    def backward(dZ, cache):
        A_prev, W, b = cache
        m = A_prev.shape[1]
        dW = 1.0/m * dZ.dot(A_prev.T)
        db = 1.0/m * np.sum(dZ,axis=1,keepdims=True)
        dA_prev = W.T.dot(dZ)
        return dA_prev, dW, db
    @staticmethod
    def backwardAct(dA, cache, activation):
        linear_cache, activation_cache = cache
        if activation == "relu":
            dZ = relu_backward(dA, activation_cache)
            dA_prev, dW, db = deepfuzzy.backward(dZ, linear_cache)
        elif activation == "sigmoid":
            dZ = sigmoid_backward(dA, activation_cache)
            dA_prev, dW, db = deepfuzzy.backward(dZ, linear_cache)
        return dA_prev, dW, db
    
    def backwardProp(self):
        L = self.L
        m = self.AL.shape[1]
        Y = self.y_train.reshape(self.AL.shape) 
        dAL = - (np.divide(Y, self.AL) - np.divide(1 - Y, 1 - self.AL))
        current_cache = self.caches[L-1]
        self.grads["dA" + str(L)], self.grads["dW" + str(L)], self.grads["db" + str(L)] = deepfuzzy.backwardAct(dAL, current_cache, "sigmoid")
        for l in reversed(range(L-1)):
            current_cache = self.caches[l]
            dA_prev_temp, dW_temp, db_temp = deepfuzzy.backwardAct(self.grads["dA"+str(l+2)], current_cache, "relu")
            self.grads["dA" + str(l + 1)] = dA_prev_temp
            self.grads["dW" + str(l + 1)] = dW_temp
            self.grads["db" + str(l + 1)] = db_temp

    def compute_cost(self):
        m = self.y_train.shape[1]
        Y = self.y_train
        AL = self.AL
        print(Y.shape)
        print(AL.shape)
        cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
        cost = np.squeeze(cost) 
        return cost

    def update_parameters(self,learning_rate):
        for l in range(self.L):
            self.W[str(l+1)] = self.W[str(l+1)] - learning_rate * self.grads["dW" + str(l+1)]
            self.b[str(l+1)] = self.b[str(l+1)] - learning_rate * self.grads["db" + str(l+1)]
            

    def predict(self):
            m = X.shape[1]
            n = self.L
            p = np.zeros((1,m))
            probas, caches = L_model_forward()
            for i in range(0, probas.shape[1]):
                if probas[0,i] > 0.5:
                    p[0,i] = 1
                else:
                    p[0,i] = 0
            
            #print results
            #print ("predictions: " + str(p))
            #print ("true labels: " + str(y))
            print("Accuracy: "  + str(np.sum((p == y)/m)))
                
            return p

In [19]:
m = deepfuzzy(layers,x_train,x_test,y_train,y_test)

In [20]:
m.initialize()

In [21]:
m.forwardProp()

(50, 60000)
(12, 60000)
(45, 60000)
(10, 60000)


In [12]:
learning_rate = 0.0075
num_iterations = 3000
for i in range(0, num_iterations):
    m.forwardProp()
    m.compute_cost()
    m.backwardProp
    print(m.grads)
    m.update_parameters(learning_rate)
    if print_cost and i % 100 == 0:
        print ("Cost after iteration %i: %f" %(i, cost))
    if print_cost and i % 100 == 0:
        costs.append(cost)

plt.plot(np.squeeze(costs))
plt.ylabel('cost')
plt.xlabel('iterations (per tens)')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

(50, 60000)
(12, 60000)
(45, 60000)
(10, 60000)
(10, 60000)
(10, 60000)
{}


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in log


KeyError: 'dW1'